# Handling duplicate, missing, or invalid data


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("../data/dirty_data.csv")
df.head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


The dirty_data.csv file contains wide format data from the weather API that has
been altered to introduce many common data issues that we will encounter in the wild.
It contains the following fields:

* PRCP: Precipitation in millimeters
* SNOW: Snowfall in millimeters
* SNWD: Snow depth in millimeters
* TMAX: Maximum daily temperature in Celsius
* TMIN: Minimum daily temperature in Celsius
* TOBS: Temperature at the time of observation in Celsius
* WESF: Water equivalent of snow in millimeters

In [5]:
df.describe()

/Users/user/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF
count,765.000000,577.000000,577.0,765.000000,765.000000,398.000000,11.000000
mean,5.360392,4.202773,NaN,2649.175294,-15.914379,8.632161,16.290909
std,10.002138,25.086077,NaN,2744.156281,24.242849,9.815054,9.489832
min,0.000000,0.000000,-inf,-11.700000,-40.000000,-16.100000,1.800000
25%,0.000000,0.000000,NaN,13.300000,-40.000000,0.150000,8.600000
50%,0.000000,0.000000,NaN,32.800000,-11.100000,8.300000,19.300000
75%,5.800000,0.000000,NaN,5505.000000,6.700000,18.300000,24.900000
max,61.700000,229.000000,inf,5505.000000,23.900000,26.100000,28.700000


We can use the `info()` method to see if we have any missing values and check that our
columns have the expected data types.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


Now, let's track down those null values in the columes shown above. Both` Series` and `DataFrame` objects provide
two methods to do so: `isnull()` and `isna()`.

In [13]:
contain_null = df[df["SNOW"].isna() | df["SNWD"].isna() | df["TOBS"].isna() | df["WESF"].isna() | df["inclement_weather"].isna() ]
contain_null.shape[0]

765

In [15]:
contain_null.head(10)

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
7,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
9,2018-01-05T00:00:00,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


We see that every single row contains some null data. Looking at the top 10 rows, we can see some
`NaN` values in each of these rows:

Note that inf and -inf are actually np.inf and -np.inf. Therefore, we can find the
number of rows with inf or -inf values by doing the following:

In [17]:
df[df["SNWD"].isin([-np.inf, np.inf])].shape[0]

577

This only tells us about a single column, though, so we could write a function that will use
a dictionary comprehension to return the number of infinite values per column in our
dataframe:

In [21]:
def get_inf_count(df:pd.DataFrame)->dict:
    """Find number of inf values per column"""
    return {col:df[df[col].isin([-np.inf, np.inf])].shape[0] for col in df.columns}

In [22]:
get_inf_count(df)

{'date': 0,
 'station': 0,
 'PRCP': 0,
 'SNOW': 0,
 'SNWD': 577,
 'TMAX': 0,
 'TMIN': 0,
 'TOBS': 0,
 'WESF': 0,
 'inclement_weather': 0}

Before we can decide on how to handle the infinite values of snow depth, we should look
at the summary statistics for snowfall (SNOW), which forms a big part of determining
the snow depth (SNWD).

In [33]:
pd.DataFrame({"np.inf Snow Depth": df.loc[df["SNWD"] == np.inf, "SNOW"].describe(),
              "-np.inf Snow Depth": df.loc[df["SNWD"] == -np.inf, "SNOW"].describe(),
             }).T

,count,mean,std,min,25%,50%,75%,max
np.inf Snow Depth,24.0,101.041667,74.498018,13.0,25.0,120.5,152.0,229.0
-np.inf Snow Depth,553.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


The snow depth was recorded as negative infinity when there was snow; however, we can't be sure this isn't just a coincidence going forward.

If we are just going to be
working with this fixed date range, we can treat that as having a depth of 0 or NaN because
it didn't snow. Unfortunately, we can't really make any assumptions with the positive
infinity entries. They most certainly aren't that, but we can't decide what they should be,
so it's probably best to leave them alone or not look at this column:

We are working with a year's data yet we have 765 rows. We should check why

In [35]:
df.describe(include="object")

,date,station,inclement_weather
count,765,765,408
unique,324,2,2
top,2018-07-05T00:00:00,GHCND:USC00280907,False
freq,8,398,384


Lets see if we can find any duplicates. Given there are 324 unique dates, and 2 unique stations.
We might assume that each day have two entries. One for each station, however that would only amount to 648 records. Which is still short of 765.


Lets see whether we can find any duplicated data

In [40]:
df[df.duplicated(keep="first")].shape

(284, 10)

Depending on what we are trying to acheive, we can handle duplication in different manners.
The rows that are returned can be modified with the `keep` arguement. By default it is `first`,
and for each row that is present more than once, we will get only the additional row

In [41]:
df[df.duplicated(keep=False)].shape

(482, 10)

In [46]:
df[df.duplicated(["date", "station"])].shape

(284, 10)

We can see that when `date` and `station` are duplicated, so is the rest of the data because the same results are before.

In [47]:
df[df.duplicated()].head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True


From the above we can see that some rows are repeated at least 3 times. 

Now that we know to find problems in our data, we shall proceed to learning different approaches to fix them. 

## Mitigating the issues


We are in an unsatisfactory state with our data, and while we can work to make it better,
the best plan of action isn't always evident. Perhaps the easiest thing we can do when faced
with this class of data issues is to remove the duplicate rows. However, it is crucial that we
evaluate the ramifications such a decision may have on our analysis.

Since we know that both stations will be for New York City, we may decide to drop the
`station` column—they may have just been collecting different data. If we then decide
to remove duplicate rows using the date column and keep the data for the station that
wasn't ?, in the case of duplicates, we will lose all data we have for the WESF column
because the ? station is the only one reporting `WESF` measurements:


In [53]:
df[df.WESF.notna()].station.unique()

array(['?'], dtype=object)

We can see that only the station `?` reports the `WESF`. 


In [80]:
#1. Perform type conversion on the date column
df.date = pd.to_datetime(df.date)

In [81]:
#2. Save WESF as a series
station_qm_wesf = df[df["station"] == "?"].drop_duplicates("date").set_index("date")["WESF"]
station_qm_wesf

date
2018-09-04   NaN
2018-09-10   NaN
2018-09-05   NaN
2018-09-01   NaN
2018-09-02   NaN
              ..
2018-06-12   NaN
2018-06-11   NaN
2018-06-10   NaN
2018-06-09   NaN
2018-06-08   NaN
Name: WESF, Length: 232, dtype: float64

In [82]:
#3. Sort the dataframe by the station column in descending order to put the station with no ID (?) last:

df.sort_values("station", ascending=False, inplace=True)

In [83]:
#4. Drop duplicate based on date column, keep the first occurence
df_deduped = df.drop_duplicates("date")

In [84]:
#5. Drop the station column and set the index to the date column (so that it matches the WESF data):

df_deduped = df_deduped.drop(columns="station").set_index("date").sort_index()
df_deduped

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True
...,...,...,...,...,...,...,...,...
2018-12-27,0.0,0.0,-inf,5.6,-2.2,-1.1,NaN,False
2018-12-28,11.7,0.0,-inf,6.1,-1.7,5.0,NaN,False
2018-12-29,21.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


Update the WESF column using the `combine_first()` method to coalesce the values to the first
non-null entry; this means that if we had data from both `stations`, we would first
take the value provided by the station with an ID, and if (and only if) that station
was null would we take the value from the station without an ID (?). Since both
`df_deduped` and `station_qm_wesf` are using the date as the index, the values
are properly matched to the appropriate date:

In [87]:
#6. 
df_deduped = df_deduped.assign(WESF=lambda x:x.WESF.combine_first(station_qm_wesf))
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


In [88]:
df_deduped.shape

(324, 8)

### **`Handling missing values NaN`**

To drop all the rows with any null data (this doesn't have to be true for all the columns of the row, so be careful), we use the `dropna()` method; in our case, this leaves us with just
4 rows:

In [96]:
df_deduped.dropna().shape

(4, 8)

In [97]:
df_deduped.dropna(how="all").shape

(324, 8)

Say we wanted to look at snow data; we would most likely want to make sure that our data had values for `SNOW`, `SNWD`, and `inclement_weather`. This can be achieved with the subset argument:

In [98]:
df_deduped.dropna(how="all", subset=["SNOW", "SNWD", "inclement_weather"]).shape

(293, 8)

Note that this operation can also be performed along columns, and that we can provide a threshold for the number of null values to be observed to drop the data with the `thresh` arguement.

For example, if we say that at least 75% of the rows must be
null to drop the column, we will drop the` WESF` column:For example, if we say that at least 75% of the rows must be null to drop the column, we will drop the` WESF` column:

In [101]:
df_deduped.columns

Index(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'WESF',
       'inclement_weather'],
      dtype='object')

In [100]:
df_deduped.dropna(axis="columns",
                 thresh=df_deduped.shape[0]*0.75).columns

Index(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'inclement_weather'], dtype='object')

Since we have a lot of null values, we will likely be more interested in keeping these values,
and perhaps finding a better way to represent them. If we replace the null data, we must
exercise caution when deciding what to fill in instead; filling in all the values we don't have
with some other value may yield strange results later on, so we must think about how we
will use this data first.

The `WESF` column contains mostly null values,
but since it is a measurement in milliliters that takes on the value of NaN when there is
no water equivalent of snowfall, we can fill in the nulls with zeros. Note that this can be
done in-place (again, as a general rule of thumb, we should use caution with in-place
operations):

In [104]:
df_deduped.loc[:, "WESF"].fillna(0, inplace=True)

In [115]:
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


In [116]:
# Handle extreme tempparatures
df_deduped = df_deduped.assign(
    TMAX = lambda x:x["TMAX"].replace(5505.0, np.nan), 
    TMIN = lambda x:x["TMIN"].replace(-40.0, np.nan)
)

In [118]:
# Assume there wont drastic changes in temperature from day. This is a big assumption
df_deduped.assign(
    TMAX = lambda x:x["TMAX"].fillna(method="ffill"), 
    TMIN = lambda x:x["TMIN"].fillna(method="ffill")
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.9,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


If we want to handle the nulls and infinite values in the`SNWD` column, we can use
the `np.nan_to_num()` function; it turns NaN into 0 and inf/-inf into very large
positive/negative finite numbers, making it possible for machine learning models

In [119]:
df_deduped.assign(SNWD=lambda x: np.nan_to_num(x.SNWD)).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-1.797693e+308,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-1.797693e+308,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-1.797693e+308,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,1.797693e+308,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,1.797693e+308,-4.4,-13.9,-13.9,0.0,True


This doesn't make much sense for our use case though. For instances of -np.inf, we
may choose to set SNWD to 0 since we saw there was no snowfall on those days. However,
we don't know what to do with np.inf, and the large positive numbers, arguably, make
this more confusing to interpret:


Depending on the data, we may decide to use `clip()`.
The `clip()` method makes it possible to cap values at a specific minimum and/or maximum threshold.

In [122]:
df_deduped.assign(SNWD = lambda x: x["SNWD"].clip(0, x["SNOW"])).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,0.0,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,0.0,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,229.0,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,127.0,-4.4,-13.9,-13.9,0.0,True


We can combine imputation with the `fillna()` method. As an example, let's fill in the
NaN values for TMAX and TMIN with their medians and TOBS with the average of TMIN
and TMAX (after imputing them):

In [123]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,14.4,5.6,10.0,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,14.4,5.6,10.0,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


In [126]:
df_deduped.apply(
    lambda x:x.fillna(x.rolling(7, min_periods=0).median())
).head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.30,-16.1,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.40,-13.9,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-6.35,-15.0,-12.75,19.3,True
2018-01-05,14.2,127.0,inf,-4.40,-13.9,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.00,-15.6,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.70,-17.2,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.80,-16.7,-8.30,0.0,False
2018-01-10,0.0,0.0,-inf,5.00,-7.8,-7.80,0.0,False


Another way of imputing missing data is to have pandas calculate what the values should
be with the `interpolate()` method. By default, it will perform linear interpolation,
making the assumption that all the rows are evenly spaced. Our data is daily data,
although some days are missing, so it is just a matter of reindexing first. Let's combine this
with the `apply()` method to interpolate all of our columns at once:

In [132]:
df_deduped.reindex(
    pd.date_range("2018-01-01", "2018-12-31", freq="D")
).apply(lambda x:x.interpolate()).head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.10,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.90,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.90,-13.60,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.90,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.0,-15.60,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.7,-17.20,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.8,-16.70,-8.30,0.0,False
2018-01-09,0.0,0.0,-inf,-1.4,-12.25,-8.05,0.0,NaN
2018-01-10,0.0,0.0,-inf,5.0,-7.80,-7.80,0.0,False


Check out January 9th, which we didn't have previously—the values for TMAX, TMIN,
and TOBS are the average of the values for the day prior (January 8th) and the day after
(January 10th):